# Creating my Proper Nouns Sets
In this notebook, I ran all the processes to extract, sort and filter proper nouns from my text. Many of these processes have been exported to the module noun_sorting, some for reuseability and some largely just to reduce clutter.  
  
*Note: During this process, I was not always 100% careful about not writing over code when it was things like remove a noun from one list and adding it to another. Therefore, although this notebook will run in its entirety, the results may not be exactly the same as the lists I ended up implementing, as I did not have time to redo all my previous work.*

In [1]:
import noun_sorting as ns
import preprocessing as pp
import re

In [2]:
clean_text = pp.load_data('clean_text')

In [3]:
ppns = pp.proper_nouns(clean_text, replace=False, return_nouns=True)

In [4]:
stopwords = pp.load_data('stopwords')

This is as many proper nouns as I could reasonably extract from the text with a function.

In [5]:
print(sorted(ppns))

['Abbey', 'Abbey Mill', 'Abbey Mill Farm', 'Abbey-Mill', 'Abbey-Mill Farm', 'Abbeyland', 'Abbots', 'Abydos', 'Adelaide', 'Admiral', 'Admiral Baldwin', 'Admiral Brand', 'Admiral Crawford', 'Admiral Croft', 'Admiralty', 'Agatha', 'Agathas', 'Agricultural Reports', 'Aladdin', 'Albion Place', 'Alderneys', 'Alfred', 'Alice', 'Allen', 'Allenham', 'Allenham Court', 'Allenham House', 'Allens', 'Alps', 'Amelia', 'America', 'Anderson', 'Andersons', 'Anhalt', 'Anna Weston', 'Annamaria', 'Anne', 'Anne Cox', 'Anne Elliot', 'Antigua', 'Antwerp', 'April', 'Archbishop', 'Arcturus', 'Argyle Buildings', 'Argyle Street', 'Ashworth', 'Asia Minor', 'Asp', 'Astley', 'Atlantic', 'August', 'Augusta', 'Augusta Hawkins', 'Aunt Emma', 'Avignon', 'Aylmers', 'Baddeley', 'Bahama', 'Baker Street', 'Bakewell', 'Banbury', 'Bank', 'Barnes', 'Barnet', 'Baron', 'Baron Wildenheim', 'Baronet', 'Baronetage', 'Barouche', 'Bartlett Buildings', 'Barton', 'Barton Cottage', 'Barton Cross', 'Barton Park', 'Barton Valley', 'Basil 

In [6]:
months = ('January', 'February', 'March', 'April', 'May(?!\sI)(?!\sGod)(?!\swe)(?!\sbe)(?!\sshe)(?!\sits)', 
          'June', 'July', 'August', 'September', 'October', 'November', 'December', 'JULY')

In [7]:
weekdays = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')

In [8]:
person_titles = ('Sir', 'Lord', 'Lady', 'Mr', 'Mr.', 'Mrs', 'Mrs.', 'Miss', 'Captain', 
                  'Colonel', 'Col', 'Col.', 'Admiral', 'Master', 'Mistress', 'Doctor',  'Nurse',
                 'Cousin', 'Baron', 'Count', 'Dowager Viscountess', 'Earl', 'General', 'Governor',
                 'Madam', 'Dr', 'Dr.', 'Hon.', 'Honourable', 'Right Hon.', 'Right Honourable')

In [9]:
place_terms = ('North', 'South', 'East', 'West', 'Northern', 'Southern', 'Eastern', 'Western',
         'Street', 'House', 'Park', 'Square', 'Road', 'Hotel', 'Walk', 'Estate',
        'Hall', 'Island', 'Building', 'Church', 'Lane', 'Cottage', 'Lower', 'Upper', 'Abbey', 'Lodge', 
         'Place', 'Court', 'Hill', 'Yard', 'Room', 'Wood', 'Mall', '- - shire',
        'Bridge', 'Mount', 'Common', 'Inn', 'Mill', 'Farm', 'Cross', 'Valley', 'Head', 'Cliff',
        'Forest', 'Castle', 'Down', 'Gardens', 'Crescent', 'Lake', 'Parsonage', 'Seas')


This particular cell won't actually be relevant again in this notebook. Later, when feeding text into a model, I use this set of people and place terms to determine whether or not to decapitalize tokens. 

In [10]:
keep_upper = {*person_titles, *place_terms, 'God', 'Heaven', 'Heavens'}
keep_upper.remove('- - shire')
pp.save_data(keep_upper, 'keep_upper')

```ProperNouns``` is a class that I created to store and modify the proper nouns sets I created. It has a number of attributes, but the most important is ```ppn_types```, which is a dict which the names of types keyed to sets of words identiied as belonging to that noun type. The second would be ```ppn_list```, which contains the results printed a few cells up - which hopefully contain most if not all of the proper nouns present in the corpus. This list is what I will be sorting into noun types in this notebook.

In [11]:
nouns = ns.ProperNouns(clean_text, person_titles, place_terms, proper_nouns_list=ppns)

This ```add``` function take the name of a noun type and an optional string or list of strings as arguments. If the noun type is already present in the class dictionary, this function will update it with the second arg. If the noun type is not present, the function will create a dict entry for the given type, and initialize it with either an empty set, or the second arg, depending on whether a second arg was given.

In [12]:
nouns.add('months', months)
nouns.add('weekdays', weekdays)
nouns.add('people')
nouns.add('places')

```sort_basic_ppns``` is probably the biggest single chunk of sorting I was able to do in one go. What this does, is it takes the people and place terms passed into the ```ProperNouns``` object on initialization, and iterates over the strings in its ```ppn_list``` attribute, looking for any strings that contain any of people and place terms. It adds these strings to the correct entry in ```ppn_types``` and collects the strings minus the term used to find it. Next, it passes through the list again looking for any strings that contain the names it found on the first pass.  

For example: on the first pass, the model used the person term ```'Mr.'``` to identify ```'Mr. Bennet'```, ```'Mr. Darcy'```, and ```'Mr. Bingley'``` as people names. Now, the funstion subtracts ```'Mr.'```, and collects the names ```Bennet```, ```Darcy``` and ```Bingley``` to use as search terms for the next pass. Also on the first pass, the function used ```Park``` to identify ```Netherfield Park``` and ```Mansfield Park``` as places. Again, it collects these terms minus the search terms, and adds ```Netherfield``` and ```Mansfield``` to the list of terms of search for on the second iteration. So then on the second pass, it will identify ```Elizabeth Bennet``` and ```Caroline Bingley``` as people, and ```Netherfield House``` and ```Mansfield Hall``` as places.

In this second loop, the function also compares each string to the list of months and weekdays passed earlier. Finally, it checks the string against a list of common English place name endings, such as ```shire``` and ```ton```. If the string has satisified not of these comparisons on this second pass, the function adds it to a ```not_classified``` list in order to separate what terms still need sorting.

In [13]:
nouns.sort_basic_ppns()

Most of these look pretty good, they're almost all place terms.

In [14]:
print(nouns.ppn_types['places'])

{'Pall Mall', 'Mansfield Parsonage', 'Claverton Down', 'Mickleham', 'Honiton', 'West Indian', 'Lower Rooms', 'Northamptonshire', 'West Indies', 'Lakes', 'Netherfield House', 'Pump Room', 'Kingsweston', 'Abbey-Mill Farm', 'South Wales', 'Salisbury', 'Upper Lyme', 'Kympton', 'Twickenham', 'Purvis Lodge', 'Marlborough Buildings', 'Crown Inn', 'Cobham', 'Rivers Street', 'Derbyshire', 'Sotherton Court', 'Barton', 'York', 'Pemberley', 'Kellynch', 'Kellynch Lodge', 'Pemberley House', 'Wiltshire', 'Highbury', 'Uppercross Hall', 'Conduit Street', 'Grosvenor', 'Oxford', 'Haggerston', 'Shropshire', 'Lansdown Hill', 'Lyme', 'Ecclesford', 'Brock Street', 'Gracechurch Street', 'Rosings', 'Staffordshire', 'Great House', 'Wimpole Street', 'Newbury', 'Everingham', 'Thornton', 'Rumford', 'Clifton', 'Octagon Room', 'Longbourn House', 'Broadway Lane', 'Lansdown Crescent', 'Magdalen Bridge', 'Pump Yard', 'Kensington Gardens', 'Argyle Buildings', 'Allenham Court', 'Eton', 'Gloucestershire', 'East Indies', '

This isn't quite as good, it has a few weird terms like ```Mansfield Park Mr``` and ```Wednesday Miss Lucas```, but aside from those, most of the errors are due to things like one character referring to another as ```Miss Somebody``` or ```Mr. Impudence```.

In [15]:
print(nouns.ppn_types['people'])

{'General Tilney', 'Admiral Croft', 'Nicholls', 'Mrs. Eltons', 'Morland', 'Merchant Taylors', 'Margaret Fraser', 'Mr. Ferrars', 'William Price', 'Mrs. Richardson', 'Mr. Richard', 'Mrs. Admiral Maxwell', 'Bragge', 'Louisa Musgrove', 'Lady Mary Maclean', 'Miss Taylor', 'Miss Coxes', 'Miss Augusta', 'Mr. Henry Tilney', 'Miss Smith', 'Sir John Middleton', 'Elton', 'Miss Jane Bates', 'Robert', 'Miss Andrews', 'John Saunders', 'Mr. George', 'Marianne', 'Miss Nash', 'Miss Lucases', 'Miss Musgroves', 'Julia', 'Edmund', 'Lady Frasers', 'Lord Morton', 'Mrs. Smallridge', 'Mr. Thomas', 'Harry', 'Captain Frederick Tilney', 'Sir Frederick', 'Miss Harville', 'Caroline Bingley', 'Brown', 'Miss Martin', 'Miss Morland', 'Mr. Wingfield', 'Jane Fairfax', 'Mr. Repton', 'Mr. Dixon', 'Willoughbys', 'Mrs Wallis', 'Master Harry', 'Whitakers', 'William', 'Charles Anderson', 'Lady Middleton', 'John Smith', 'Harris', 'Miss Crawford', 'Mr Smith', 'Hilll', 'George Wickham', 'Lord Macartney', 'Mrs. Fraser', 'Sir Lew

In [16]:
print('Number of total proper nouns found initially:', len(ppns), 
      '\nNumber of nouns that still haven\'t been classified after this function:', 
      len(nouns.not_classified))

Number of total proper nouns found initially: 1370 
Number of nouns that still haven't been classified after this function: 468


Almost 2/3 of the data has been classified!

In [17]:
print(sorted(nouns.not_classified))

['Abbey', 'Abbeyland', 'Abbots', 'Abydos', 'Adelaide', 'Admiralty', 'Agatha', 'Agathas', 'Agricultural Reports', 'Aladdin', 'Alderneys', 'Alfred', 'Alice', 'Alps', 'Amelia', 'America', 'Anhalt', 'Annamaria', 'Antigua', 'Antwerp', 'Archbishop', 'Arcturus', 'Ashworth', 'Asia Minor', 'Asp', 'Astley', 'Atlantic', 'Avignon', 'Aylmers', 'Baddeley', 'Bahama', 'Bakewell', 'Bank', 'Barnes', 'Barnet', 'Baron', 'Baronet', 'Baronetage', 'Barouche', 'Bateses', 'Bear', 'Beavers', 'Belinda', 'Bell', 'Bella', 'Belle', 'Belmont', 'Bermuda', 'Betsey', 'Betty', 'Bishop', 'Blair', 'Blues', 'Boarding-school', 'Bonomi', 'Boulanger', 'Braithwaites', 'Bristol', 'British', 'Broadwood', 'Building', 'Butler', 'Cambridge', 'Camilla', 'Canopus', 'Cape', 'Captain', 'Caractacus', 'Cardinal Wolsey', 'Cartwright', 'Casino', 'Cassino', 'Cassiopeia', 'Castle', 'Cecilia', 'Ceremonies', 'Chamberlayne', 'Channel', 'Chaperon', 'Charlotte', 'Charlotte Davis', 'Charmouth', 'China', 'Christchurch', 'Christian', 'Christians', '

And now comes the most tedious part of this process. First, I looked through the list of ```not_classified``` terms to see which ones were easily identifiable as a person or place. I also looked through the ```people``` and ```places``` lists to see if there were any obvious misclassifications. If I were to do this again, I think at this point instead of doing it manually, I would bring in a list of common names and a list of English towns and places, which maybe would catch a lot of what I did manually.

In [18]:
not_found_persons = {'Elinor', 'Kitty', 'Betsey', 'Jemima', 'Ladyship', 'Dawson', 'Adelaide', 'Agatha',
                    'Sophia', 'Charlotte', 'Flora', 'Johnson', 'Douglas', 'Charlotte Davis', 'Janet',
                    'Agathas', 'Alfred', 'Alice', 'Amelia', 'Annamaria', 'Belinda', 'Bella', 'Betty',
                    'Cardinal Wolsey', 'Cecilia', 'Christopher Jackson', 'Collinses', 'Dorothy', 'Dr',
                    'Eleanor', 'Emily', 'Farmer Mitchell', 'Hannah', 'Henrietta', 'Ibbotsons', 
                    'Isabella', 'Jackson', 'Jacksons', 'Mackenzie', 'Maddison', 'Martha Sharpe', 'Matilda',
                    'Nancy', 'Oliver', 'Olivers', 'Penelope', 'Philipses', 'Rebecca', 'Robin Adair', 
                    'Sally', 'Sam', 'Sandersons', 'Sarah', 'Sophy', 'Sophys', 'Susan', 'Theodore', 'Tom',
                    'Tupman', 'Tupmans'}

not_found_places = ('Gibraltar', 'Switzerland', 'Ramsgate', 'America', 'Europe', 'Bahama', 'Russia',
                   'United Kingdoms', 'Thames', 'China', 'Pembroke', 'Asia Minor', 'Atlantic',
                   'Italy', 'Enscombe', 'Cambridge', 'Alps', 'Kent', 'Westminster', 'Cornwall',
                   'Charmouth', 'Dove Dale', 'Dublin', 'Gloucester', 'Hartfield', 'Lisbon', 'London',
                   'Liverpool', 'Longbourn-house', 'Madeira', 'Maple Grove', 'Newcastle', 'Newfoundland',
                   'Newmarket', 'Norfolk', 'Peterborough', 'Portsmouth', 'Reading', 'Scarborough',
                   'Surry', 'Sussex', 'Trafalgar', 'Tunbridge', 'Tuscany', 'Wakefield')

to_remove_people = ('Gretna Green', 'London Lydia', 'Manager', 'Impudence', 'Longtown', 'Merchant Taylors',
                    'Miss Somebody', 'Miss Woodhouse Miss Taylor', 'St Domingo', 'St. George', 
                    'St. George Fields', 'St. James', 'Mansfield Park Mr', 'Wednesday Miss Lucas',
                   'Mrs Rushworth Maria Bertram')

to_remove_places = ('Church', 'South', 'End', 'House', 'Buildings')

In [19]:
nouns.add('people', not_found_persons)
nouns.add('places', not_found_places)
nouns.remove('people', to_remove_people)
nouns.remove('places', to_remove_places)
    

In [20]:
not_found_list = sorted(list(nouns.not_classified))

In [21]:
print(not_found_list)

['Abbey', 'Abbeyland', 'Abbots', 'Abydos', 'Admiralty', 'Agricultural Reports', 'Aladdin', 'Alderneys', 'Anhalt', 'Antigua', 'Antwerp', 'Archbishop', 'Arcturus', 'Ashworth', 'Asp', 'Astley', 'Avignon', 'Aylmers', 'Baddeley', 'Bakewell', 'Bank', 'Barnes', 'Barnet', 'Baron', 'Baronet', 'Baronetage', 'Barouche', 'Bateses', 'Bear', 'Beavers', 'Bell', 'Belle', 'Belmont', 'Bermuda', 'Bishop', 'Blair', 'Blues', 'Boarding-school', 'Bonomi', 'Boulanger', 'Braithwaites', 'Bristol', 'British', 'Broadwood', 'Building', 'Buildings', 'Butler', 'Camilla', 'Canopus', 'Cape', 'Captain', 'Caractacus', 'Cartwright', 'Casino', 'Cassino', 'Cassiopeia', 'Castle', 'Ceremonies', 'Chamberlayne', 'Channel', 'Chaperon', 'Christchurch', 'Christian', 'Christians', 'Christmas', 'Christmas Day', 'Church', 'Cleopatra', 'Cleveland', 'Col', 'Colonel', 'Columella', 'Combe', 'Combe Magna', 'Commerce', 'Commissioner', 'Constantia', 'Cottage', 'Cottager', 'Count', 'Courier', 'Court', 'Cowper', 'Cowper Tirocinium', 'Coxe', 

Same thing again. This time, based on examination of the nouns, I realized I needed to add additional categories of literature and boats. Literature to catch things like ```Lady of the Lake``` and ```Romance of the Forest``` and boats because boats are named like people (sometimes) but contextually treated as places.

In [22]:
add_places = {'Abbeyland', 'Ashworth', 'Avignon', 'Bakewell', 'Barnet', 'Belmont', 'Bermuda', 
              'Bristol', 'Cape', 'Channel', 'Christchurch', 'Cleveland', 'Combe', 'Combe Magna',
             'Constantia', 'Crewkherne', 'Cromer', 'Cumberland', 'Dawlish', 'Dresden', 'Dugdale',
             'Epsom', 'Exeter', 'Garrison', 'Hatfield', 'Holborn', 'Holyhead', 'Huntingdon',
             'Lessingby', 'Longstaple', 'Longtown', 'Mansion-house', 'Marlborough', 'Mediterranean', 
              'Merchant Taylors', 'Midland', 'Minehead', 'Molland', 'North', 'Oriel', 'Parliament',
             'Petty France', 'Plymouth', 'Priory', 'Putney', 'Pyrenees', 'Richmond', 'Save', 'Sicily',
             'Sidmouth', 'Sound', 'South', 'Spithead', 'St Domingo', 'St. Anthony', 'St. Clement',
             'St. George', 'St. James', 'St. Paul', 'Stanhill', 'Stoke', 'Streights', 'Tattersall',
             'Temple', 'Texel', 'Theatre', 'Turner', 'Weymouth', 'Whitwell', 'Wick Rocks', 'Windsor',
             'Winthrop', 'Somerset', 'Gretna Green', 'St. George Fields', 'Chatsworth', 'Dovedale', 
              'the Peak'}
add_people = {'Abbots', 'Admiralty', 'Archbishop', 'Aylmers', 'Baddeley', 'Barnes', 'Bateses', 'Belle',
             'Braithwaites', 'Camilla', 'Caractacus', 'Agricola', 'Cartwright', 'Chamberlayne',
             'Cottager', 'Coxe', 'Dick Jackson', 'Doge', 'Durands', 'Ellis', 'Freeman', 'Garrick',
             'Gentleman', 'Georgiana', 'Gibson', 'Goldsmith', 'Gouldings', 'Gregorys', 'Harringtons', 
             'Biddy Henshawe', 'Hetty', 'Julius Caesar', 'Laurentina', 'Milmans', 'Mitchell', 'Mitchells',
             'Nanny', 'Neptune', 'Norval', 'Parrys', 'Patty', 'Pen', 'Pooles', 'Pope', 'Prior', 'Pug',
             'Ross', 'Scholey', 'Selina', 'Serle', 'Severus', 'Sewell', 'Skinners', 'Spicers',
             'St. Aubin', 'Stephen', 'Sterne', 'Thompson', 'Wallises', 'Wells', 'Wilcox', 'Wright',
              'James the Second', 'Queen Mab'}
literature = {'Hamlet', 'Othello', 'Macbeth', 'Shylock', 'Douglas', 'Crabbe\'s Tales', 'the Idler',
             'Lover\'s Vows', 'the Gamester', 'Richard III', 'Tom Jones', 'the Monk', 'Shakespeare',
             'the Vicar of Wakefield', 'the Romance of the Forest', 'the Children of the Abbey',
             'Gamester', 'Fordyce Sermons', 'Agricultural Reports', 'Baronetage', 'Blair', 'Butler',
             'Henry the Eighth', 'Necromancer of the Black Forest', 'Midnight Bell', 'Orphan of the Rhine',
             'Horrid Mysteries', 'Mysterious Warnings', 'Marmion', 'Lady of the Lake', 'Giaour', 
              'Bride of Abydos', 'the Rivals', 'School for Scandal', 'Wheel of Fortune', 'Heir at Law',
             'Vicar of Wakefield', 'Castle of Wolfenbach', 'Clermont', 'Thomson', 'Cowper', 'Scott', 
             'Mr Scott', 'Lord Byron', 'Udolpho'}
boats = {'Indiaman', 'Laconia', 'Cleopatra', 'Antwerp', 'Endymion', 'H.M. Sloop Thrush', 'Asp', 'Thrush',
        'Canopus', 'Elephant', 'Grappler'}

In [23]:
nouns.add('people', add_people)
nouns.add('places', add_places)
nouns.add('literature', literature)
nouns.add('boats', boats)

At this point, many of the words left in the ```not_classified``` list are not easy to identify as a person or place at a glance, so I needed some textual context. The function ```show_context``` prints the text surrounding the first occurrance of each term in the argument list. If keyword arg ```findall=True```, the function will print the context for every occurrance of that word in the text. There is also an additional optional argument ```context_width``` to widen the range of context provided, and arguments ```left_regex, right_regex, prefix, suffix```, which allow more customization of the search regex expression (default is ```'\W{word_to_find}\W'```.

In [24]:
nouns.show_context(nouns.not_classified)

MOUNT :

zy, and Kitty, said Mrs. Bennet, to walk to Oakham Mount this morning. It is a nice long walk, and Mr. Darc
METHODISTS :

 as a celebrated preacher in some great society of Methodists, or as a missionary into foreign parts. She tried 
COL :

 - only think what fun! Not a soul knew of it, but Col. and Mrs. Forster, and Kitty and me, except my aun
TAN :

sleek, well-tied parcels of, Men Beavers and, York Tan were bringing down and displaying on the counter, 
LATIN :

 it but an old man playing at see-saw and learning Latin; upon my soul there is not. This critique, the jus
IRELAND :

 case is, you see, that the Campbells are going to Ireland. Mrs. Dixon has persuaded her father and mother to
CHRISTMAS :

rowd, but of that I despair. I sincerely hope your Christmas in Hertfordshire may abound in the gaieties which 
SCANDAL :

ven the tragedians; and the Rivals, The School for Scandal, Wheel of Fortune, Heir at Law, and a long et cete
SCOTCH :

erto, was glad to purchase praise an

e; and being disappointed in my hope of seeing the Marquis of Longtown and General Courteney here, some of my
BAROUCHE :

, for a week; and as Dawson does not object to the Barouche box, there will be very good room for one of you -
MISTRESSES :

some topic, and to wander at large amongst all the Mistresses and Misses of Highbury, and their card-parties. Sh
LANE :

ng within view of the lodges opening into Hunsford Lane, in order to have the earliest assurance of it; an
PARENTS :

nts of, A most desirable Estate in South Wales; To Parents and Guardians; and a, Capital seasond Hunter. Fann
CHRISTMAS DAY :

t. The weather was most favourable for her; though Christmas Day, she could not go to church. Mr. Woodhouse would h
EIGHTH :

d before since I was fifteen. I once saw Henry the Eighth acted, or I have heard of it from somebody who did
MRS :

lied that he had not. But it is, returned she; for Mrs. Long has just been here, and she told me all abou
COMMERCE :

ascertain that they both li

will get! - Good gracious! (giggling as she spoke) Id lay my life I know what my cousins will say, when 
GENERAL :

 in the army. He has the promise of an ensigncy in General -LNM- regiment, now quartered in the North. It is 
SHYLOCK :

ndertake any character that ever was written, from Shylock or Richard III down to the singing hero of a farce
QUARTERLY REVIEWS :

 time as they could with sofas, and chit-chat, and Quarterly Reviews, till the return of the others, and the arrival of
CASTLE :

r names directly; here they are, in my pocketbook. Castle of Wolfenbach, Clermont, Mysterious Warnings, Necr
NATURE :

orrected me without it. Do you? - I have no doubt. Nature gave you understanding: - Miss Taylor gave you pri
UPPER :

on; the woody varieties of the cheerful village of Upper Lyme; and, above all, Pinny, with its green chasms
SIR :

what an establishment it would be for one of them. Sir William and Lady Lucas are determined to go, merel
BUILDINGS :

, lately arrived at their cousi

 danger, was the winter that I passed by myself at Deal, when the Admiral (Captain Croft then) was in the 
CASSINO :

g little or doing less, Lady Middleton sat down to Cassino, and as Marianne was not in spirits for moving abo
MRS RUSHWORTH MARIA BERTRAM :

NOT FOUND

EPICURISM :

- not sorry to be driven by the observation of his Epicurism, his selfishness, and his conceit, to rest with co
INDIAMAN :

ce in London, and the other midshipman on board an Indiaman. But though she had seen all the members of the fa
COLUMELLA :

 pursuits, employments, professions, and trades as Columella. They will be brought up, said he, in a serious ac
GRECIAN :

eauty, to gain a distant eminence; where, from its Grecian temple, her eye, wandering over a wide tract of co
HON :

 a year, if the match takes place. The lady is the Hon. Miss Morton, only daughter of the late Lord Morto
GOTHIC :

the sun playing in beautiful splendour on its high Gothic windows. But so low did the building stand, that s
CAPT

In [25]:
add_people = {'Anhalt', 'Baronet', 'Boulanger', 'Goddard', 'Bonomi', 'Columella', 'Hymen', 'Valancourt',
              'Dick', 'Fletcher', 'Folly', 'Montoni'}
add_places = {'Blues', 'Broadwood', 'Comissioner', 'Deal', 'Dorking', 'Exeter Exchange', 'Lombardy', 
             'St. Mark\'s Place', 'Matlock', 'Motherbank', 'Astley', 'Devizes', 'Randalls', 'Randall', 
              'Woolwich'}
nouns.remove('people','Henry the Eighth')
found = ('Antigua', 'Butler', 'Blair', 'Baronetage', 'Asp', 'Idler', 'Isle', 'Wight', 'Mark', 'Abydos',
        'Canopus', 'Elephant', 'Grappler', 'Marmion', 'Giaour', 'Monk', 'Peak')
nouns.add('people', add_people)
nouns.add('places', add_places)
nouns.remove('not classified', found)

In [26]:
nouns.show_context(nouns.not_classified)

MOUNT :

zy, and Kitty, said Mrs. Bennet, to walk to Oakham Mount this morning. It is a nice long walk, and Mr. Darc
METHODISTS :

 as a celebrated preacher in some great society of Methodists, or as a missionary into foreign parts. She tried 
COL :

 - only think what fun! Not a soul knew of it, but Col. and Mrs. Forster, and Kitty and me, except my aun
TAN :

sleek, well-tied parcels of, Men Beavers and, York Tan were bringing down and displaying on the counter, 
LATIN :

 it but an old man playing at see-saw and learning Latin; upon my soul there is not. This critique, the jus
IRELAND :

 case is, you see, that the Campbells are going to Ireland. Mrs. Dixon has persuaded her father and mother to
CHRISTMAS :

rowd, but of that I despair. I sincerely hope your Christmas in Hertfordshire may abound in the gaieties which 
SCANDAL :

ven the tragedians; and the Rivals, The School for Scandal, Wheel of Fortune, Heir at Law, and a long et cete
SCOTCH :

erto, was glad to purchase praise an

very bright. Yes, and the Bear. I wish I could see Cassiopeia. We must go out on the lawn for that. Should you b
END :

siness, my dear, your spending the autumn at South End instead of coming here. I never had much opinion o
VOWS :

tated to try their skill. The play had been Lovers Vows, and Mr. Yates was to have been Count Cassel. A tr
VINGT-UN :

have enabled them to ascertain that they both like Vingt-un better than Commerce; but with respect to any othe
DOCTOR :

 cried Mrs. Jennings; very pretty, indeed! and the Doctor is a single man, I warrant you. There now, said Mi
TOWER :

bling in St. George Fields, the Bank attacked, the Tower threatened, the streets of London flowing with blo
PAPA :

 till Kitty runs away. I am not going to run away, Papa, said Kitty, fretfully; if I should ever go to Bri
BISHOP :

me; and after THAT, as soon as he can light upon a Bishop, he will be ordained. I wonder what curacy he will
NORTHERN :

Meryton. The time fixed for the beginning of their Nor

s wherever he went, and how full the Master of the Ceremonies ball had been; and she went through it very well, 
EASTER :

 on the subject, for having received ordination at Easter, I have been so fortunate as to be distinguished b
FORDYCE SERMONS :

ere produced, and after some deliberation he chose Fordyce Sermons. Lydia gaped as he opened the volume, and before h
JOVE :

work for her. Edmund smiled and shook his head. By Jove! this wont do, cried Tom, throwing himself into a 
RIVALS :

ng that could satisfy even the tragedians; and the Rivals, The School for Scandal, Wheel of Fortune, Heir at
SPANISH :

ls behind the house, and of the beautiful oaks and Spanish chestnuts which were scattered over the intermedia
SEPT :

t it did not contain a denial. Gracechurch Street, Sept. 1. MY DEAR NIECE, I have just received your lette
GOWLAND :

using any thing in particular? No, nothing. Merely Gowland, he supposed. No, nothing at all. Ha! he was surpr
E :

NOT FOUND

GARDENS :

o beautiful a

In [27]:
nouns.add('people', {'de Bourgh', 'Sir Lewis de Bourgh', 'Miss de Bourgh',
               'Lady Catherine de Bourgh', 'Right Honourable Lady Catherine de Bourgh',
              'the King', 'Rev. Philip Elton', 'Rev. Mr. Norris', 'Broadwood',
              'Lady of Branxholm Hall', 'Marquis', 'F. C. Weston Churchill',
              'E. GARDINER', 'EDW. GARDINER', 'M. GARDINER'})
nouns.add('places', {'House of Commons', 'Isle of Wight', 'B.', 'Blenheim', 'Warwick', 'Kenelworth', 'the North',
              'the South', '(?<!north )Wiltshire', 'Cork', 'France', 'Ireland', 'Italy'})
nouns.add('literature', {'The Mirror', 'History of England'})
nouns.add('boats', {'H.M.S. Thrush', 'H.M.S. Antwerp', 'Great Nation'})

In [28]:
nouns.remove('people',['Sir Lewis', 'Right Honourable Lady Catherine', 'Marquis', 'Mistress'])
nouns.remove('places', ['Broadwood', 'Stilton', 'Wiltshire', 'Great Nation', 'Indian'])

In [29]:
#Oxford, Blenheim, Warwick, Kenelworth, Birmingham, Haggerston
nouns.show_context(nouns.not_classified)

MOUNT :

zy, and Kitty, said Mrs. Bennet, to walk to Oakham Mount this morning. It is a nice long walk, and Mr. Darc
METHODISTS :

 as a celebrated preacher in some great society of Methodists, or as a missionary into foreign parts. She tried 
COL :

 - only think what fun! Not a soul knew of it, but Col. and Mrs. Forster, and Kitty and me, except my aun
TAN :

sleek, well-tied parcels of, Men Beavers and, York Tan were bringing down and displaying on the counter, 
LATIN :

 it but an old man playing at see-saw and learning Latin; upon my soul there is not. This critique, the jus
CHRISTMAS :

rowd, but of that I despair. I sincerely hope your Christmas in Hertfordshire may abound in the gaieties which 
SCANDAL :

ven the tragedians; and the Rivals, The School for Scandal, Wheel of Fortune, Heir at Law, and a long et cete
SCOTCH :

erto, was glad to purchase praise and gratitude by Scotch and Irish airs, at the request of her younger sist
SECOND :

s chapel was fitted up as you see it, 

very bright. Yes, and the Bear. I wish I could see Cassiopeia. We must go out on the lawn for that. Should you b
END :

siness, my dear, your spending the autumn at South End instead of coming here. I never had much opinion o
VOWS :

tated to try their skill. The play had been Lovers Vows, and Mr. Yates was to have been Count Cassel. A tr
VINGT-UN :

have enabled them to ascertain that they both like Vingt-un better than Commerce; but with respect to any othe
DOCTOR :

 cried Mrs. Jennings; very pretty, indeed! and the Doctor is a single man, I warrant you. There now, said Mi
TOWER :

bling in St. George Fields, the Bank attacked, the Tower threatened, the streets of London flowing with blo
PAPA :

 till Kitty runs away. I am not going to run away, Papa, said Kitty, fretfully; if I should ever go to Bri
BISHOP :

me; and after THAT, as soon as he can light upon a Bishop, he will be ordained. I wonder what curacy he will
NORTHERN :

Meryton. The time fixed for the beginning of their Nor

work for her. Edmund smiled and shook his head. By Jove! this wont do, cried Tom, throwing himself into a 
STILTON :

end Cole, and that she was come in herself for the Stilton cheese, the north Wiltshire, the butter, the celer
RIVALS :

ng that could satisfy even the tragedians; and the Rivals, The School for Scandal, Wheel of Fortune, Heir at
SPANISH :

ls behind the house, and of the beautiful oaks and Spanish chestnuts which were scattered over the intermedia
SEPT :

t it did not contain a denial. Gracechurch Street, Sept. 1. MY DEAR NIECE, I have just received your lette
GOWLAND :

using any thing in particular? No, nothing. Merely Gowland, he supposed. No, nothing at all. Ha! he was surpr
E :

NOT FOUND

GARDENS :

o beautiful a Sunday as to draw many to Kensington Gardens, though it was only the second week in March. Mrs.
HALL :

s the very utmost that they can have lived at West Hall; and how they got their fortune nobody knows. They
H :

 years ago from the Mediterranean by Wi

I identified a few more noun types to add. I chose these because they are all used within sentences in a way that is distinct from all existing noun types.

In [30]:
nationalities = {'French', 'Irish', 'Italian', 'Scottish|Scotch', 'German', 'Grecian', 'British', 
                 'West Indian', 'Indian'}
languages = {'(?:(?<=in\s)|(?<=learned\s)|(?<=taught\sher\s)|(?<=;\s))French', 'Italian(?=\slines)'}
holidays = {'Christmas', 'Christmas Day', 'Easter', 'Easter-day', 'Mid-summer', 'Midsummer', 'Michaelmas'}

In [31]:
nouns.add('nationalities', nationalities)
nouns.add('languages', languages)
nouns.add('holidays', holidays)

In [32]:
caps = [word for word in re.findall('(?:[A-Z]{2,}\s*)+', clean_text) if word.lower().strip() not in stopwords]

In [33]:
print(caps)

['CAROLINE BINGLEY', 'DEAR SIR', 'WILLIAM COLLINS', 'FITZWILLIAM DARCY', 'PLC', 'PLC', 'PLC', 'PLC', 'PLC', 'MY DEAR HARRIET', 'LYDIA BENNET', 'PLC', 'MY DEAR SIR', 'MY DEAR BROTHER', 'EDW', 'GARDINER', 'LNM', 'GARDINER', 'PLC', 'LNM', 'PLC', 'MY DEAR NIECE', 'PLC', 'GARDINER', 'PLC', 'DEAR SIR', 'MY DEAR LIZZY', 'LET ', 'DRAW HIM IN', 'CATCHING ', 'MIND ', 'SOMETIMES ', 'ROBERT ', 'OCCASION ', 'ESTEEM ', 'MY DEAR MADAM', 'JOHN WILLOUGHBY', 'MONTH', 'YOUR WORD', 'TWICE ', 'ALL PARTIES', 'HAS BEEN ', 'NOT ELINOR', 'LOOK ', 'GAUCHERIE ', 'TRIED ', 'LOOKED ', 'ROBERTS ', 'EDWARD ', 'ROBERT ', 'DEAR SIR', 'LUCY FERRARS', 'FAITH ', 'THE END  ', 'CHARADE', 'WINDSOR', 'JULY', 'MY DEAR MADAM', 'WESTON CHURCHILL', 'FINIS  ', 'ELLIOT OF KELLYNCH HALL', 'II', 'III ', 'XIV', 'THE END  ']


In [34]:
caps = set(caps[:45])
# caps.remove()

The original search function checked for a single capital letter followed by one or more lower case letters. So let's check out what the stuff in all caps is.

In [35]:
[caps.discard(word) for word in ['ALL PARTIES', 'CATCHING ', 'CHARADE', 'DEAR SIR', 'DRAW HIM IN', 'ESTEEM ',
 'FAITH ', 'FINIS  ', 'GAUCHERIE ', 'HAS BEEN ', 'JULY', 'MONTH', 'MY DEAR BROTHER', 'MY DEAR HARRIET',
                                'MY DEAR LIZZY', 'MY DEAR MADAM', 'MY DEAR NIECE', 'MY DEAR SIR', 'NOT ELINOR',
 'OCCASION ','SOMETIMES ', 'THE END  ', 'TRIED ', 'TWICE ']]
print(caps)

{'CAROLINE BINGLEY',
 'EDW',
 'FITZWILLIAM DARCY',
 'GARDINER',
 'JOHN WILLOUGHBY',
 'LET ',
 'LNM',
 'LOOK ',
 'LYDIA BENNET',
 'MIND ',
 'PLC',
 'ROBERT ',
 'WILLIAM COLLINS',
 'YOUR WORD'}

In [37]:
nouns.add('people', {'CAROLINE BINGLEY', 'FITZWILLIAM DARCY', 'JOHN WILLOUGHBY', 'LUCY FERRARS', 'LYDIA BENNET',
              'WILLIAM COLLINS', 'Mrs. EDWARD Ferrars', 'Mrs. ROBERT Ferrars', 'Mr. ROBERT Ferrars', 
              'ELLIOT'})
nouns.add('places', {'KELLYNCH HALL', 'WINDSOR'})

In [38]:
nouns.add('people', {'Charles Smith, Esq. Tunbridge Wells', 'Clara Partridge', 'Flora Ross', 
               'John Willoughby', 'Old Allen', 'Wm. Elliot', 'William Walter Elliot', 'Walter Elliot', 
                     'Hon. Miss Morton'})
nouns.add('places', {'Antigua', 'Baly-craig', 'Bartlett Building', 'East Kingham Farm', 'White-Hart'})
nouns.add('literature', {'Cowper Tirocinium', 'Cramer'})

In [39]:
nouns.ppn_types['people']

{'Abbots',
 'Adelaide',
 'Admiral',
 'Admiral Baldwin',
 'Admiral Brand',
 'Admiral Crawford',
 'Admiral Croft',
 'Admiralty',
 'Agatha',
 'Agathas',
 'Agricola',
 'Alfred',
 'Alice',
 'Allen',
 'Allens',
 'Amelia',
 'Anderson',
 'Andersons',
 'Anhalt',
 'Anna Weston',
 'Annamaria',
 'Anne',
 'Anne Cox',
 'Anne Elliot',
 'Archbishop',
 'Augusta',
 'Augusta Hawkins',
 'Aunt Emma',
 'Aylmers',
 'Baddeley',
 'Barnes',
 'Baron Wildenheim',
 'Baronet',
 'Basil Morley',
 'Bates',
 'Bateses',
 'Belinda',
 'Bella',
 'Belle',
 'Bennet',
 'Bennets',
 'Benwick',
 'Bertram',
 'Bertrams',
 'Betsey',
 'Betty',
 'Biddy Henshawe',
 'Bingley',
 'Bingleys',
 'Bird',
 'Bonomi',
 'Boulanger',
 'Bragge',
 'Bragges',
 'Braithwaites',
 'Brandon',
 'Broadwood',
 'Brown',
 'CAROLINE BINGLEY',
 'Camilla',
 'Campbell',
 'Campbells',
 'Captain Benwick',
 'Captain Brigden',
 'Captain Carter',
 'Captain Frederick Tilney',
 'Captain Frederick Wentworth',
 'Captain Harville',
 'Captain Hunt',
 'Captain Marshall',
 'C

In [40]:
nouns.show_context(nouns.ppn_types['people'])

GENERAL TILNEY :

your name, and you have a right to know his. It is General Tilney, my father. Catherine answer was only, Oh! - but i
ADMIRAL CROFT :

e very first application for the house was from an Admiral Croft, with whom he shortly afterwards fell into company
NICHOLLS :

 ball, it is quite a settled thing; and as soon as Nicholls has made white soup enough I shall send round my c
THOMPSON :

, -And waste its fragrance on the desert air. From Thompson, that - -It is a delightful task -To teach the you
MRS. ELTONS :

e nobody. Emma did not want to be classed with the Mrs. Eltons, the Mrs. Perrys, and the Mrs. Coles, who would fo
MORLAND :

 been. THE END  No one who had ever seen Catherine Morland in her infancy would have supposed her born to be 
MRS. EDWARD FERRARS :

taking up some work from the table, to inquire for Mrs. EDWARD Ferrars. She dared not look up; - but her mother and Maria
MARGARET FRASER :

the endless questions I shall have to answer! Poor Margaret Fraser will 

 in a volume some dozen lines of Milton, Pope, and Prior, with a paper from the Spectator, and a chapter fr
JOHN SMITH :

lice, and publishing the banns of marriage between John Smith and Mary Brown, he could conceive nothing more rid
HARRIS :

 serious than Elinor, now looked very grave on Mr. Harris report, and confirming Charlotte fears and caution
MISS CRAWFORD :

in the brother and sister of Mrs. Grant, a Mr. and Miss Crawford, the children of her mother by a second marriage. 
MR SMITH :

ce. From his wife account of him she could discern Mr Smith to have been a man of warm feelings, easy temper, 
HILLL :

ay. Lydia, my love, ring the bell. I must speak to Hilll, this moment. It is not Mr. Bingley, said her husb
GEORGE WICKHAM :

o, Miss Eliza, I hear you are quite delighted with George Wickham! - Your sister has been talking to me about him, a
WALTER ELLIOT :

 in the perfect happiness of the union. FINIS  Sir Walter Elliot, of Kellynch Hall, in Somersetshire, was a man who
LORD 

s wife. I shall always have a great regard for the Miss Martins, especially Elizabeth, and should be very sorry to
ELTONS :

est things do take place! When Miss Smiths and Mr. Eltons get acquainted - they do indeed - and really it is
NANCY :

e hour end to another. Lord! here comes your beau, Nancy, my cousin said tother day, when she saw him cross
SIR LEWIS DE BOURGH :

ight Honourable Lady Catherine de Bourgh, widow of Sir Lewis de Bourgh, whose bounty and beneficence has preferred me to 
THOMAS :

ewspapers announced to the world, that the lady of Thomas Palmer, Esq. was safely delivered of a son and hei
ELIZABETH BENNET :

ment, by his having slighted one of her daughters. Elizabeth Bennet had been obliged, by the scarcity of gentlemen, to
MRS. MARTIN :

icity which could speak with so much exultation of Mrs. Martin having two parlours, two very good parlours, indee
FITZWILLIAM :

ent at last, that she recollected having heard Mr. Fitzwilliam Darcy formerly spoken of as a very prou

e me think of them directly. People of the name of Tupman, very lately settled there, and encumbered with ma
MISS ELIZA BENNET :

eading to cards? said he; that is rather singular. Miss Eliza Bennet, said Miss Bingley, despises cards. She is a great
SIR THOMAS BERTRAM :

en thousand pounds, had the good luck to captivate Sir Thomas Bertram, of Mansfield Park, in the county of Northampton, 
TAYLOR :

-nothing fellow! I have no patience with him. Mrs. Taylor told me of it half an hour ago, and she was told i
CAPTAINS HARVILLE :

ed around Mrs Musgrove, and Charles came back with Captains Harville and Wentworth. The appearance of the latter could 
MR MUSGROVES :

uch truly sympathising friend as Lady Russell. The Mr Musgroves had their own game to guard, and to destroy, their
MR. BINGLEY :

better, for as you are as handsome as any of them, Mr. Bingley might like you the best of the party. My dear, you
SIR LEWIS DE BOURGH :

of what the glazing altogether had originally cost Sir Lewis De 

 for I have a great many more than I can ever use. William Larkins let me keep a larger quantity than usual this year
HYMEN :

at he was sure at this rate it would be May before Hymen saffron robe would be put on for us. Oh! the pains
MISS SNEYDS :

them, and found them on the pier: Mrs. and the two Miss Sneyds, with others of their acquaintance. I made my bow 
ROSE :

nt bear to see them dirty and nasty. Now there Mr. Rose at Exeter, a prodigious smart young man, quite a b
LIZZY :

; though I must throw in a good word for my little Lizzy. I desire you will do no such thing. Lizzy is not 
KING :

 two next. Then, the two third he danced with Miss King, and the two fourth with Maria Lucas, and the two 
GARDINERS :

ng with her, without any danger of seeing him. The Gardiners stayed a week at Longbourn; and what with the Phil
MARTINS :

nversation - and but for her acquaintance with the Martins of Abbey-Mill Farm, it must have been the whole. B
SEVERUS :

ded the other; and of the Roman 

...

I was to give place to Lord St Ives, and a certain Admiral Baldwin, the most deplorable-looking personage you
HURSTS :

 have Mr. Bingley chaise to go to Meryton; and the Hursts have no horses to theirs. I had much rather go in 
REV. MR. NORRIS :

years, found herself obliged to be attached to the Rev. Mr. Norris, a friend of her brother-in-law, with scarcely any
MRS. HILLL :

We have heard nothing from town. Dear madam, cried Mrs. Hilll, in great astonishment, dont you know there is an 
HARRIET SMITH :

onger dreaded by the fair mistress of the mansion. Harriet Smith was the natural daughter of somebody. Somebody had
DASHWOOD :

he means of uniting them. * * * * *  The family of Dashwood had long been settled in Sussex. Their estate was 
MRS. CLARKE :

l you any thing if you ask. You see I cannot leave Mrs. Clarke. It was lucky, however, for Mrs. Jennings curiosit
MISS MADDOXES :

ct what it was that she had heard about one of the Miss Maddoxes, or what it was that Lady Prescott had n

? Yes, I have, presently. But here comes a friend, Captain Brigden; I shall only say, How dye do? as we pass, however
M. GARDINER :

n wanting me this half hour. Your, very sincerely, M. GARDINER. The contents of this letter threw Elizabeth into 
MISS SNEYD :

 to have been noticed for the next six months; and Miss Sneyd, I believe, has never forgiven me. That was bad in
COLONEL HARRISON :

ott had noticed in Fanny: she was not sure whether Colonel Harrison had been talking of Mr. Crawford or of William whe
BELINDA :

omentary shame. It is only Cecilia, or Camilla, or Belinda; or, in short, only some work in which the greates
MISS MARIA WARD :

national importance. Finis  About thirty years ago Miss Maria Ward, of Huntingdon, with only seven thousand pounds, h
BATES :

ither when Mrs. Perry drank tea with Mrs. and Miss Bates, or when Mrs. and Miss Bates returned the visit. N
AGRICOLA :

obertson, than if the genuine words of Caractacus, Agricola, or Alfred the great. You are fond of hi

 importance. You know what he thinks of Cowper and Scott; you are certain of his estimating their beauties 
MISS OTWAY :

- Mrs. Otway, I protest! - and good Mr. Otway, and Miss Otway and Miss Caroline. - Such a host of friends! - and
SANDERSONS :

cry out at once and have done with. The Parrys and Sandersons luckily are coming tonight you know, and that will
JOHN WILLOUGHBY :

le for me not to hear all. The name of Willoughby, John Willoughby, frequently repeated, first caught my attention; a
WHITAKER :

t dinner. Nothing would satisfy that good old Mrs. Whitaker, but my taking one of the cheeses. I stood out as 
FAIRFAX :

ll the Knightleys together, as she does about Jane Fairfax. One is sick of the very name of Jane Fairfax. Eve
RUSHWORTH :

gest estates and finest places in the country. Mr. Rushworth was from the first struck with the beauty of Miss 
SHEPHERD :

of his tradespeople, and the unwelcome hints of Mr Shepherd, his agent, from his thoughts. The Kellynch proper
MR. MARTI

young man in the Blues for the sake of that horrid Lord Stornaway, who has about as much sense, Fanny, as Mr. Rushwo
COLONEL FITZWILLIAM :

require them, for Mr. Darcy had brought with him a Colonel Fitzwilliam, the younger son of his uncle, Lord William and to
YATES :

ay see they are so many couple of lovers - all but Yates and Mrs. Grant - and, between ourselves, she, poor
AYLMERS :

ach. Mrs. R. has been spending the Easter with the Aylmers at Twickenham (as to be sure you know), and is not
LADY PATRONESS :

l that to me. Only give me a carte-blanche. - I am Lady Patroness, you know. It is my party. I will bring friends wi
SIR WILLIAM :

what an establishment it would be for one of them. Sir William and Lady Lucas are determined to go, merely on tha
MR. DRUMMOND :

d me there was a very beautiful set of pearls that Mr. Drummond gave his daughter on her wedding-day and that Miss
MISS WILLIAMS :

ou, maam? said almost every body. Yes; it is about Miss Williams, I am sure. And who is 

ss Woodhouse, and defying Miss Smith. The charming Augusta Hawkins, in addition to all the usual advantages of perfec
CHARLOTTE LUCAS :

t might turn her mother thoughts, now asked her if Charlotte Lucas had been at Longbourn since her coming away. Yes, 
OLD ALLEN :

gue; it was broken by Thorpe saying very abruptly, Old Allen is as rich as a Jew - is not he? Catherine did not


In [41]:
nouns.show_context(nouns.ppn_types['places'])

BAKEWELL :

be here till to-morrow; and indeed, before we left Bakewell, we understood that you were not immediately expec
PALL MALL :

ble! Where did you hear it? In a stationer shop in Pall Mall, where I had business. Two ladies were waiting for
MANSFIELD PARSONAGE :

house I shall call to mind the conjugal manners of Mansfield Parsonage with respect. Even Dr. Grant does show a thorough 
HONITON :

 on horseback, do you? added Sir John. No. Only to Honiton. I shall then go post. Well, as you are resolved t
SURRY :

ey, from having been longer than usual absent from Surry, were exciting of course rather more than the usua
WEST INDIAN :

eafter useful to Sir Thomas in the concerns of his West Indian property? No situation would be beneath him; or wh
LOWER ROOMS :

g nobody at all. They made their appearance in the Lower Rooms; and here fortune was more favourable to our heroi
ALPS :

 counties of England, was to be looked for. Of the Alps and Pyrenees, with their pine forests and their

ard business, my dear, your spending the autumn at South End instead of coming here. I never had much opinion o
ABBEY-MILL :

- and but for her acquaintance with the Martins of Abbey-Mill Farm, it must have been the whole. But the Martins
HIGH STREET :

ere rattled into a narrow street, leading from the High Street, and drawn up before the door of a small house now
ABBEYLAND :

 Sir John new plantations at Barton Cross, and the Abbeyland; and we will often go to the old ruins of the Prio
HOLBORN :

me tell it to Lucy, for I think of going as far as Holborn to-day. No, maam, not even Lucy if you please. One
NEWTON :

sider, here are the two Miss Careys come over from Newton, the three Miss Dashwoods walked up from the cotta
BAHAMA :

 was in the West Indies. We do not call Bermuda or Bahama, you know, the West Indies. Mrs Musgrove had not a
DUBLIN :

 directly, and they would give them the meeting in Dublin, and take them back to their country seat, Baly-cr
MANSFIELD PARK :

 the good l

y home; I have an establishment at my own house in Woodston, which is nearly twenty miles from my father, and 
MOOR PARK :

ittle worth the trouble of gathering. Sir, it is a Moor Park, we bought it as a Moor Park, and it cost us - tha
BARTON CROSS :

go on; we will walk to Sir John new plantations at Barton Cross, and the Abbeyland; and we will often go to the ol
KINGHAM FARM :

made a little purchase within this half year; East Kingham Farm, you must remember the place, where old Gibson use
WESTMINSTER :

ical time of his life? If you had only sent him to Westminster as well as myself, instead of sending him to Mr. P
LANSDOWN ROAD :

u? Yes. Well, I saw him at that moment turn up the Lansdown Road, driving a smart-looking girl. Did you indeed? Did
ALLENHAM :

om the cottage, along the narrow winding valley of Allenham, which issued from that of Barton, as formerly des
MANSION HOUSE :

rehending only two days, was spent entirely at the Mansion House; and she had the satisfaction of kn

or, was invited to spend a week with his sister in Northamptonshire before he went to sea. Their eager affection in me
LAKES :

l carry us, said Mrs. Gardiner, but perhaps to the Lakes. No scheme could have been more agreeable to Eliza
MEDITERRANEAN :

next summer, when I had still the same luck in the Mediterranean. And I am sure, Sir, said Mrs Musgrove, it was a l
READING :

 Palmer and Colonel Brandon would get farther than Reading that night. Elinor, however little concerned in it
ABBEY-MILL FARM :

- and but for her acquaintance with the Martins of Abbey-Mill Farm, it must have been the whole. But the Martins occu
SOUTH WALES :

ious advertisements of, A most desirable Estate in South Wales; To Parents and Guardians; and a, Capital seasond 
SALISBURY :

e country; not but what we have very good shops in Salisbury, but it is so far to go - eight miles is a long wa
WINTHROP :

 he steps into very pretty property. The estate at Winthrop is not less than two hundred and fifty acres, b

 the usual terms; how it had been first settled in Cheshire; how mentioned in Dugdale, serving the office of h
NEWFOUNDLAND :

, Henry, with the friends of his solitude, a large Newfoundland puppy and two or three terriers, was ready to rece
NEW LONDON INN :

. They was stopping in a chaise at the door of the New London Inn, as I went there with a message from Sally at the 
NORTH SEAS :

, when the Admiral (Captain Croft then) was in the North Seas. I lived in perpetual fright at that time, and had
WEST INDIA :

fair than heretofore, by some recent losses on his West India estate, in addition to his eldest son extravagance
(?<!NORTH )WILTSHIRE :

ef of the property about Fullerton, the village in Wiltshire where the Morlands lived, was ordered to Bath for 
WINDSOR :

 made him ill. Yours ever, A. W. [To Mrs. Weston.] WINDSOR-JULY. MY DEAR MADAM, If I made myself intelligible
PYRENEES :

 of England, was to be looked for. Of the Alps and Pyrenees, with their pine forests and their vices

GIBRALTAR :

with your poor brother. I always forgot. It was at Gibraltar, mother, I know. Dick had been left ill at Gibralt
EASTON :

 Mansfield Wood, and Edmund took the copses beyond Easton, and we brought home six brace between us, and mig
CREWKHERNE :

ir, while you were at dinner; and going on now for Crewkherne, in his way to Bath and London. Elliot! Many had l
NORTHAMPTON :

homas Bertram, of Mansfield Park, in the county of Northampton, and to be thereby raised to the rank of a baronet
PUTNEY :

 Fullerton, the envy of every valued old friend in Putney, with a carriage at her command, a new name on her
THAMES :

told us at Taunton. The Baronet will never set the Thames on fire, but there seems to be no harm in him. - r
WEST HALL :

alf is the very utmost that they can have lived at West Hall; and how they got their fortune nobody knows. They
NORTH :

cy in General -LNM- regiment, now quartered in the North. It is an advantage to have it so far from this pa
WHITE HOUSE :

ttled

Aaaand a couple more noun categories. These ones are more to do with the end product than the text itself. I thought it would be weird to ask for a place name and then have text print out that's like 'He sailed across the London', or 'They visited the very small neighboring town called Russia'

In [42]:
countries = {'America', 'Antigua', 'Asia Minor', 'China', 'East Indies', 'West Indies', 'Europe',
            'France', 'Great Britain', 'Ireland', 'Italy', 'Isle of Wight', 'Russia', 'Sicily',
            'South Wales', 'Switzerland', 'United Kingdoms', 'Bahama', 'Venice', 'Western Islands'}
waters = {'Atlantic', 'Channel', 'Cape', 'North Seas', 'Streights'}


In [43]:
nouns.add('countries', countries)
nouns.add('waters', waters)

In [44]:
nouns.remove('places', ['West India', 'West Indian', 'Cheapside', 'Island'])

In [45]:
nouns.ppn_types['places']

{'(?<!north )Wiltshire',
 'Abbey Mill',
 'Abbey Mill Farm',
 'Abbey-Mill',
 'Abbey-Mill Farm',
 'Abbeyland',
 'Albion Place',
 'Allenham',
 'Allenham Court',
 'Allenham House',
 'Alps',
 'America',
 'Antigua',
 'Argyle Buildings',
 'Argyle Street',
 'Ashworth',
 'Asia Minor',
 'Astley',
 'Atlantic',
 'Avignon',
 'B.',
 'Bahama',
 'Baker Street',
 'Bakewell',
 'Baly-craig',
 'Banbury',
 'Barnet',
 'Bartlett Building',
 'Bartlett Buildings',
 'Barton',
 'Barton Cottage',
 'Barton Cross',
 'Barton Park',
 'Barton Valley',
 'Bath',
 'Bath Street',
 'Beachey Head',
 'Bedford',
 'Bedford Square',
 'Beechen Cliff',
 'Belmont',
 'Berkeley Street',
 'Berkeley Streets',
 'Bermuda',
 'Birmingham',
 'Black Forest',
 'Blaize Castle',
 'Blenheim',
 'Blues',
 'Bond Street',
 'Box Hill',
 'Branxholm Hall',
 'Brighton',
 'Bristol',
 'Broad Street',
 'Broadway Lane',
 'Brock Street',
 'Brockham',
 'Brunswick Square',
 'Cambridge',
 'Camden Place',
 'Cape',
 'Channel',
 'Charmouth',
 'Chatsworth',
 'Chea

Now I decided that I wanted to leave words like ```Street``` and ```Park``` in the text, since they add context. So I filtered these out and replaced some. Some of these terms I put back in with regex expressions, because of things like: ```Edward Street``` is a place, but I don't want the model to identify ```Edward``` as a place nouns. Similarly, ```Great House``` is a place, but ```Great``` also appears in sentences like ```Great news from London has just arrived``` and so shouldn't be tagged as a place in and of itself. 

In [46]:
compound_places = nouns.find_compound_places()

In [47]:
compound_places

{'Pall Mall': 'Pall',
 'Mansfield Parsonage': 'Mansfield',
 'Lower Rooms': 'Lower',
 'Pump Room': 'Pump',
 'Octagon Room': 'Octagon',
 'Broadway Lane': 'Broadway',
 'Magdalen Bridge': 'Magdalen',
 'Pump Yard': 'Pump',
 'Argyle Buildings': 'Argyle',
 'Western Islands': 'Western',
 'High Street': 'High',
 'Mansfield Park': 'Mansfield',
 'Stanwix Lodge': 'Stanwix',
 'Rosing Park': 'Rosing',
 'Hanover Square': 'Hanover',
 'Bartlett Buildings': 'Bartlett',
 'Sackville Street': 'Sackville',
 'Pulteney Street': 'Pulteney',
 'Berkeley Street': 'Berkeley',
 'Drury Lane': 'Drury',
 'Berkeley Streets': 'Berkeley',
 'Lucas Lodge': 'Lucas',
 'Westgate Buildings': 'Westgate',
 'Bond Street': 'Bond',
 'Box Hill': 'Box',
 'Walcot Church': 'Walcot',
 'Abbey Mill Farm': 'Abbey  Farm',
 'Moor Park': 'Moor',
 'Kingham Farm': 'Kingham',
 'Mansion House': 'Mansion',
 'Union Street': 'Union',
 'Mansfield Common': 'Mansfield',
 'Park Street': 'Park',
 'Concert Room': 'Concert',
 'Brunswick Square': 'Brunswick

In [48]:
compound_places['Parsonage House'] = 'Parsonage'
compound_places['Abbey Mill Farm'] = 'Abbey Mill'
del compound_places['House of Commons']

In [49]:
nouns.replace_compound_places(compound_places)

And now, I decided I wanted to sort the names in ```people``` into male, female, and last names to make both for better context for sentence generation and to make for better output.

In [50]:
nouns.find_people_with_titles()

{'General Tilney': 'Tilney',
 'Admiral Croft': 'Croft',
 'Mrs. Eltons': 'Eltons',
 'Mrs. EDWARD Ferrars': 'EDWARD Ferrars',
 'Mr. Ferrars': 'Ferrars',
 'Mrs. Richardson': 'Richardson',
 'Mr. Richard': 'Richard',
 'Mrs. Admiral Maxwell': 'Admiral Maxwell',
 'Lady Mary Maclean': 'Mary Maclean',
 'Miss Taylor': 'Taylor',
 'Miss Coxes': 'Coxes',
 'Miss Augusta': 'Augusta',
 'Mr. Henry Tilney': 'Henry Tilney',
 'Miss Smith': 'Smith',
 'Sir John Middleton': 'John Middleton',
 'Miss Jane Bates': 'Jane Bates',
 'Miss Andrews': 'Andrews',
 'Mr. George': 'George',
 'Miss Nash': 'Nash',
 'Miss Lucases': 'Lucases',
 'Miss Musgroves': 'Musgroves',
 'Lady Frasers': 'Frasers',
 'Lord Morton': 'Morton',
 'Mrs. Smallridge': 'Smallridge',
 'Mr. Thomas': 'Thomas',
 'Captain Frederick Tilney': 'Frederick Tilney',
 'Sir Frederick': 'Frederick',
 'Miss Harville': 'Harville',
 'Miss Martin': 'Martin',
 'Miss Morland': 'Morland',
 'Mr. Wingfield': 'Wingfield',
 'Mr. Repton': 'Repton',
 'Mr. Dixon': 'Dixon',
 

In [51]:
unclassified_people_with_titles = nouns.sort_people_names()

In [52]:
unclassified_people_with_titles

{'Honourable John Yates': 'John Yates',
 'Mrs. Rushworth Maria Bertram': 'Rushworth Maria Bertram'}

In [53]:
[print(nouns.ppn_types[name_type]) for name_type in ['last_names', 'male_names', 'female_names']]

{'Nicholls', 'Arthur', 'Jeffereys', 'Morland', 'Coles', 'Martin', 'Prescott', 'Courteney', 'Maclean', 'Croft', 'Watson', 'Bragge', 'Godby', 'Bird', 'Sneyds', 'Cooper', 'Larolles', 'Elton', 'Brigden', 'Robinson', 'Wildenheim', 'Wingfield', 'Cassel', 'Harrison', 'Gray', 'Thorpes', 'Graham', 'Annesley', 'Holford', 'Shirley', 'Ives', 'Cole', 's Harville', 'Harding', 'Jones', 'Millar', 'Rooke', 'Atkinson', 'Harry', 'Ravenshaw', 'Bertrams', 'Drew', 'Brown', 'Impudence', 'Hurst', 'Smith', 'Elliot', 'Wallis', 'Eleanors', 'Clarke', 'Donavan', 'Perrys', 'Hughes', 'Bickerton', 'de Bourgh', 'Speed', 'Middletons', 'Musgroves', 'Partridge', 'Smallridge', 'Cox', 'Lucas', 'Prince', 'Harris', 'Hilll', 'Monday', 'Frasers', 'Tilney', 'Scott', 'Rushworth', 'Fairfax', 'Radcliffe', 'Bridgets', 'Shepherd', 'Whitaker', 'Rose', 'King', 'Prices', 'Weston', 'Martins', 'Grey', 'Simpson', 'Green', 'Jefferies', 'Harville', 'Owen', 'Maxwell', 'Hunt', 'Palmer', 'Russell', 'Dashwoods', 'George', 'Crawford', 'Steeles',

[None, None, None]

In [54]:
nouns.unclassified_people()

['Kings',
 'Coles',
 'Sneyds',
 'Browne',
 'Thorpes',
 'Coxe',
 'Bertrams',
 'Smith,',
 'Perrys',
 'Middletons',
 'Musgroves',
 'Frasers',
 'Prices',
 'Martins',
 'Dashwoods',
 'Steeles',
 'Elliott',
 'Hayters',
 'Owens',
 'Eltons',
 'Bennets',
 'Williams',
 'Westons',
 'Smiths',
 'Knightleys']

In [55]:
nouns.unclassified_people(partial_match=False)

['Goddard',
 'Thompson',
 'Georgiana',
 'Kitty',
 'Archbishop',
 'Harringtons',
 'Philipses',
 'Gentleman',
 'Bateses',
 'M.',
 'Susan',
 'Folly',
 'Bella',
 'Robin',
 'Alfred',
 'Sarah',
 'Serle',
 'Abbots',
 'Johnson',
 'Emily',
 'Charlotte',
 'Biddy',
 'BENNET',
 'Adelaide',
 'Baronet',
 'Parrys',
 'Annamaria',
 'Douglas',
 'Broadwood',
 'Queen',
 'Mab',
 'Henrietta',
 'Dorothy',
 'Oliver',
 'Matilda',
 'Pen',
 'Olivers',
 'CAROLINE',
 'Betty',
 'Hannah',
 'Eleanor',
 'Ellis',
 'De',
 'Wright',
 'Ross',
 'Jemima',
 'E.',
 'Sally',
 'FITZWILLIAM',
 'Prior',
 'Hymen',
 'Sandersons',
 'Garrick',
 'Severus',
 'Isabella',
 'Columella',
 'Jacksons',
 'Laurentina',
 'Elinor',
 'Collinses',
 'Maddison',
 'Belle',
 'Mitchell',
 'Jackson',
 'Scholey',
 'WILLIAM',
 'LYDIA',
 'Camilla',
 'de',
 'LUCY',
 'Martha',
 'Amelia',
 'Sir',
 'Bonomi',
 'Nanny',
 'Ibbotsons',
 'Cartwright',
 'Tupmans',
 'Cottager',
 'Milmans',
 'WILLOUGHBY',
 'Barnes',
 'Davis',
 'Admiralty',
 'Chamberlayne',
 'Caractacu

In [56]:
nouns.add('female_names', {'Eleanor', 'Isabella', 'Charlotte', 'Georgiana', 'Clara', 'Betsey', 'Jemima',
                    'Biddy', 'Elinor', 'Flora', 'Folly', 'Adelaide', 'Sophia', 'CAROLINE', 'Belle',
                    'Amelia', 'Henrietta', 'Cecilia', 'LYDIA', 'Emily', 'Camilla', 'LUCY', 'Bella', 
                     'Agatha', 'Maddison', 'Alice', 'Nancy', 'Belinda', 'Betty', 'Laurentina', 'Sarah',
                    'Kitty', 'Penelope', 'Hannah', 'Rebecca', 'Matilda', 'Sally', 'Selina', 'Annamaria',
                    'Pen', 'Sophy', 'Martha', 'Biddy', 'Patty', 'Dorothy', 'Susan', 'Hetty', 'Nanny', 
                     'the Queen', 'Mab', 'Ellis'})
nouns.add('male_names',{'Dick', 'Tom', 'EDWARD', 'Philip', 'EDW.', 'Theodore', 'WILLIAM', 'Stephen', 'Alfred',
                  'Christopher', 'FITZWILLIAM', 'Oliver', 'Severus', 'Sam', 'JOHN', 'Aubin', 'Anhalt', 
                   'Lewis', 'Farmer', 'Chamberlayne', 'Gibson', 'Scholey', 'Valancourt', 'Norval', 
                   'Mackenzie', 'Cottager', 'the King', 'Doge', 'Robin', 'Butler', 'the Butler', 'the Count',
                  'the Archbishop', 'The Baronet', 'Columella', 'Cartwright'})
nouns.add('last_names',{'Morley', 'Coxe', 'Goulding', 'Saunders', 'Smith', 'GARDINER', 'Stevenson', 'Sanderson'
                  'Henshawe', 'Ross', 'Barnes', 'Dawson', 'Collinse', 'FERRARS', 'Sharpe', 'Batese', 'DARCY',
                  'Philipse', 'Wright', 'Freeman', 'BINGLEY', 'Mitchell', 'Fletcher', 'Goldsmith', 'Jackson', 
                   'BENNET', 'Abdy', 'Groom', 'Larkins', 'Parry', 'ELLIOT', 'Poole', 'Tupman', 'Saunders', 
                   'Baddeley', 'Barnes', 'Abbot', 'Davis', 'Aylmer', 'Gregory', 'Durand', 'Ibbotson', 
                   'Milman', 'Oliver', 'Braithwaite', 'Goddard', 'Adair', 'Wallise', 'Harrington', 'Wilcox', 
                   'Serle', 'Wallis', 'Spicer', 'Wolsey', 'Cromwell', 'Blair'})
nouns.add('literature', {'Udolpho', 'Douglas'})
nouns.add('places', {'Sewell', 'Buckingham', 'Broadwood'})
to_check = ['Caesar', 'Hymen',  
            'Bonomi', 'Agricola', 'Montoni']
#?? M., 'M.P.', 'M.D.' Caractacus, Agricola, or Alfred the greatMr. RobertsonMr. Hume or Mr. Robertson

Again, I decided I needed a few more categories. Most of the author names had been in people, but in context they appear in sentences like 'She was quite engrossed in her perusal of a volume of Shakespeare, which could easily make for weird sentence generation if a user input name is used there.

In [57]:
papers = {'the Courier', 'the Spectator', 'the Times', 'the Elegant Extracts', 'the Agricultural Reports'}
authors = {'Sterne', 'Lord Byron', 'Shakespeare', 'Caractacus', 'Thompson', 'Garrick', 'Agricola',
          'Alfred the great', 'Cramer', 'Cowper', 'Mr Scott', 'Scott', 'Thomson', 'Mr\. Hume', 
           'Mr\. Robertson', 'Bonomi', '(?<!Miss )Pope', 'Milton', 'Prior'}
nouns.add('papers', papers)
nouns.add('authors', authors)

In [58]:
nouns.show_context(to_check)

CAESAR :

 time have we mourned over the dead body of Julius Caesar, and to bed and not to bed, in this very room, for
HYMEN :

at he was sure at this rate it would be May before Hymen saffron robe would be put on for us. Oh! the pains
BONOMI :

dvice, and laid before me three different plans of Bonomi. I was to decide on the best of them. My dear Cour
AGRICOLA :

obertson, than if the genuine words of Caractacus, Agricola, or Alfred the great. You are fond of history! And
MONTONI :

 of wronging him. It was the air and attitude of a Montoni! What could more plainly speak the gloomy workings


In [59]:
literature_to_check = ['Blair', 'Butler','Cramer','Mr Scott','Scott','Thomson','Thompson', 'Fordyce Sermons',
                      'Mr\. Hume', 'Mr\. Robertson', 'Sterne', 'Cowper', 'Shakespeare', 'Lord Byron',
                      'Cowper Tirocinium', 'Lovers Vows', 'Agricultural Reports']

In [60]:
nouns.show_context(literature_to_check)

BLAIR :

supposing the preacher to have the sense to prefer Blair to his own, do all that you speak of? govern the c
BUTLER :

ts for Yates and Crawford, and here is the rhyming Butler for me, if nobody else wants it; a trifling part, 
CRAMER :

re is something quite new to me. Do you know it? - Cramer. - And here are a new set of Irish melodies. That,
MR SCOTT :

ey walked together some time, talking as before of Mr Scott and Lord Byron, and still as unable as before, and
SCOTT :

 importance. You know what he thinks of Cowper and Scott; you are certain of his estimating their beauties 
THOMSON :

usic enough in London to content her. And books! - Thomson, Cowper, Scott - she would buy them all over and o
THOMPSON :

, -And waste its fragrance on the desert air. From Thompson, that - -It is a delightful task -To teach the you
FORDYCE SERMONS :

ere produced, and after some deliberation he chose Fordyce Sermons. Lydia gaped as he opened the volume, and before h
MR\. HUME :

d probably 

In [61]:
nouns.add('literature', {'Fordyce\'s Sermons', 'Tirocinium', 'Lovers\' Vows'})

In [62]:
nouns.remove('literature', list(nouns.ppn_types['authors']))

In [63]:
nouns.show_context(nouns.ppn_types['places'])

BAKEWELL :

be here till to-morrow; and indeed, before we left Bakewell, we understood that you were not immediately expec
PALL MALL :

ble! Where did you hear it? In a stationer shop in Pall Mall, where I had business. Two ladies were waiting for
BROADWAY(?=\SLANE) :

years ago - that Miss Taylor and I met with him in Broadway Lane, when, because it began to drizzle, he darted
MANSFIELD PARSONAGE :

house I shall call to mind the conjugal manners of Mansfield Parsonage with respect. Even Dr. Grant does show a thorough 
UNION(?=\SSTREET) :

d Mr Elliot (always obliging) just setting off for Union Street on a commission of Mrs Clay. She now felt a
HONITON :

 on horseback, do you? added Sir John. No. Only to Honiton. I shall then go post. Well, as you are resolved t
SURRY :

ey, from having been longer than usual absent from Surry, were exciting of course rather more than the usua
LOWER ROOMS :

g nobody at all. They made their appearance in the Lower Rooms; and here fortune was more fa

hat a contrast did it offer to his last address in Rosing Park, when he put his letter into her hand! She kn
GLOUCESTERSHIRE :

g a clergyman, and of a very respectable family in Gloucestershire. With more than usual eagerness did Catherine hast
CAPE :

e first week of August, when he came home from the Cape, just made into the Grappler. I was at Plymouth dr
SOMERSET :

m into the cherished, or the prohibited, county of Somerset, for as such was it dwelt on by turns in Marianne 
COBB :

et almost hurrying into the water, the walk to the Cobb, skirting round the pleasant little bay, which, in
OCTAGON(?=\SROOM) :

they took their station by one of the fires in the Octagon Room. But hardly were they so settled, when the do
WESTERN ISLANDS :

ne and I had such a lovely cruise together off the Western Islands. Poor Harville, sister! You know how much he wante
HUNSFORD :

me filial scruples on that head, as you will hear. Hunsford, near Westerham, Kent, 11th October. DEAR SIR, The
CORK :

nc

eed not be put off. Why should not they explore to Box Hill though the Sucklings did not come? They could go t
WAKEFIELD :

 entertaining. And I know he has read the Vicar of Wakefield. He never read the Romance of the Forest, nor The 
LOMBARDY :

 screen of them altogether, interspersed with tall Lombardy poplars, shut out the offices. Marianne entered th
HIGH(?=\SSTREET) :

ere rattled into a narrow street, leading from the High Street, and drawn up before the door of a small ho
DELAFORD PARSONAGE :

 that, if I am alive, I shall be paying a visit at Delaford Parsonage before Michaelmas; and I am sure I shant go if Luc
ENSCOMBE :

 income, but still it was nothing in comparison of Enscombe: she did not cease to love her husband, but she wa
LANSDOWN :

Could it be Mr Elliot? They knew he was to dine in Lansdown Crescent. It was possible that he might stop in hi
HARTFIELD :

 and November evening must be struggled through at Hartfield, before Christmas brought the next visit from Isab


ghbury, including Randalls in the same parish, and Donwell Abbey in the parish adjoining, the seat of Mr. Kni
BLACK(?=\SFOREST) :

 Clermont, Mysterious Warnings, Necromancer of the Black Forest, Midnight Bell, Orphan of the Rhine, and Ho
CLAYTON PARK :

ss Nash, that as he was coming back yesterday from Clayton Park, he had met Mr. Elton, and found to his great surp
BRIGHTON :

t satisfaction. They are going to be encamped near Brighton; and I do so want papa to take us all there for th
BROAD(?=\SSTREET) :

 they indeed, cried Thorpe; for, as we turned into Broad Street, I saw them - does he not drive a phaeton w
THE PEAK :

ated beauties of Matlock, Chatsworth, Dovedale, or the Peak. Elizabeth was excessively disappointed; she had s
EDWARD(?=\SSTREET) :

e did not say what. She then took a large house in Edward Street, and has since maintained herself by lettin
TUNBRIDGE :

thin abundance of silver paper was a pretty little Tunbridge-ware box, which Harriet opened: it was well lined 

o know it better. The scenes in its neighbourhood, Charmouth, with its high grounds and extensive sweeps of cou
CONDUIT STREET :

s from Lady Middleton, announcing their arrival in Conduit Street the night before, and requesting the company of he
SCARBOROUGH :

esley is with her. The others have been gone on to Scarborough, these three weeks. She could think of nothing mor
GROSVENOR :

directly, and of their meaning to dine that day in Grosvenor street, where Mr. Hurst had a house. The next was 
HAGGERSTON :

is business, I will immediately give directions to Haggerston for preparing a proper settlement. There will not 
CHANNEL :

w hurried happy lines, written as the ship came up Channel, and sent into Portsmouth with the first boat that
DAWLISH :

ink, - was his next observation, in a cottage near Dawlish. Elinor set him right as to its situation; and it 
PULTENEY(?=\SSTREET) :

ch for him in vain; but at last, in returning down Pulteney Street, she distinguished him on the right han

ad of waiting for me, you took the volume into the Hermitage Walk, and I was obliged to stay till you had finished i
LAURA PLACE :

 Dalrymple had taken a house, for three months, in Laura Place, and would be living in style. She had been at Bat
LANGHAM :

I was telling you of my idea of moving the path to Langham, of turning it more to the right that it may not c
ST. MARK PLACE :

hour, and they had only accomplished some views of St. Mark Place, Venice, when Frank Churchill entered the room. Em
NORLAND COMMON :

nd I hope will in time be better. The enclosure of Norland Common, now carrying on, is a most serious drain. And the
BLAIZE CASTLE :

e able to do ten times more. Kingsweston! Aye, and Blaize Castle too, and anything else we can hear of; but here is
LOWER(?=\SROOMS) :

g nobody at all. They made their appearance in the Lower Rooms; and here fortune was more favourable to our
PUMP(?=\SYARD) :

tonished. He turned back and walked with me to the Pump Yard. He had been prevented 

eant to take her in the carriage, leave her at the Abbey Mill, while she drove a little farther, and call for he
HEREFORD :

way on Monday. We are going to Lord Longtown, near Hereford, for a fortnight. Explanation and apology are equa
ANTIGUA :

 Sir Thomas means will be rather straitened if the Antigua estate is to make such poor returns. Oh! that will
THORNBERRY :

When I found he was really going to his friends at Thornberry Park for the whole day to-morrow, I had compassion
ST. MARK'S PLACE :

NOT FOUND

TETBURY :

How long do you think we have been running it from Tetbury, Miss Morland? I do not know the distance. Her bro
UPPER ROOMS :

rtant evening came which was to usher her into the Upper Rooms. Her hair was cut and dressed by the best hand, he
BLACK FOREST :

 Clermont, Mysterious Warnings, Necromancer of the Black Forest, Midnight Bell, Orphan of the Rhine, and Horrid My
MANCHESTER STREET :

iles - nay, eighteen - it must be full eighteen to Manchester Street - was a seriou

In [64]:
# King vs Kings, Gray
#last_names > male_names, languages > nationalities
# Edward Street, Charles II, Mrs Charles

In [66]:
nouns.show_context(nouns.unclassified_people(partial_match=False))

THOMPSON :

, -And waste its fragrance on the desert air. From Thompson, that - -It is a delightful task -To teach the you
ARCHBISHOP :

as very far from dreading a rebuke either from the Archbishop, or Lady Catherine de Bourgh, by venturing to danc
GENTLEMAN :

 his nephew and niece, and their children, the old Gentleman days were comfortably spent. His attachment to the
JOHNSON :

 of a week, Fanny was tempted to apply to them Dr. Johnson celebrated judgment as to matrimony and celibacy, 
BARONET :

ear, in spite of what they told us at Taunton. The Baronet will never set the Thames on fire, but there seems
DOUGLAS :

ain. Neither Hamlet, nor Macbeth, nor Othello, nor Douglas, nor the Gamester, presented anything that could s
BROADWOOD :

 Bates, was, that this pianoforte had arrived from Broadwood the day before, to the great astonishment of both 
DE :

ne herself says that in point of true beauty, Miss De Bourgh is far superior to the handsomest of her se
PRIOR :

 in a volume some

In [67]:
[print(nouns.ppn_types[name_type]) for name_type in ['last_names', 'male_names', 'female_names']]

{'Nicholls', 'Arthur', 'Goddard', 'Jeffereys', 'Morland', 'Coles', 'Martin', 'Prescott', 'Courteney', 'Saunders', 'Maclean', 'Croft', 'Watson', 'Bragge', 'Godby', 'Bird', 'Sneyds', 'Serle', 'Cooper', 'Larolles', 'Elton', 'Brigden', 'Robinson', 'Wildenheim', 'Wingfield', 'Cassel', 'Harrison', 'Gray', 'BENNET', 'Thorpes', 'Graham', 'Annesley', 'Coxe', 'Holford', 'Wallise', 'Shirley', 'Ives', 'Cole', 's Harville', 'Harding', 'Jones', 'SandersonHenshawe', 'Millar', 'Braithwaite', 'Rooke', 'Atkinson', 'Oliver', 'Harry', 'Ravenshaw', 'Bertrams', 'Drew', 'Brown', 'Abdy', 'Groom', 'Impudence', 'Harrington', 'Hurst', 'Smith', 'Elliot', 'Goulding', 'Wallis', 'Eleanors', 'Clarke', 'Donavan', 'Perrys', 'Wright', 'Hughes', 'Bickerton', 'de Bourgh', 'Speed', 'Middletons', 'Ross', 'Musgroves', 'Partridge', 'Smallridge', 'Cox', 'Lucas', 'Prince', 'Harris', 'Hilll', 'Monday', 'Frasers', 'Tilney', 'Scott', 'Collinse', 'Rushworth', 'Fairfax', 'Radcliffe', 'Bridgets', 'Shepherd', 'Whitaker', 'Rose', 'King

[None, None, None]

In [68]:
nouns.remove('last_names', ['Bridgets','Careys', 'Eleanors', 'Lucases', 'SandersonHenshawe', 'Webbs'])
nouns.add('female_names', {'Bridget', 'Eleanor'})
nouns.add('last_names',{'Carey', 'Lucase', 'Webb', 'Sanderson', 'Henshawe'})


In [69]:
nouns.remove('last_names', ['George', 'Arthur', 'John', 'Charles', 'Thomas', 'Scott'])
nouns.add('male_names', {'George', 'Arthur', 'John', 'Charles', 'Thomas'})

In [70]:
nouns.add('literature', ['The\sChildren\sof\sthe\sAbbey', 'the\sGamester', 'the\sIdler', 'the\sMonk', 
                         'the\sRivals', 'The\sRomance\sof\sthe\sForest', 'The\sVicar\sof\sWakefield'])
nouns.add('literature', ['Children\sof\sthe\sAbbey', 'Gamester', 'Idler', 'Monk', 'Rivals',
                         'Romance\sof\sthe\sForest', 'Vicar\sof\sWakefield'])

I was more or less satisfied with my noun lists at this point, so for a final check, I called a function to see if there are any terms that appear in the set for more than one type of term.

In [73]:
del nouns.ppn_types['people']

In [74]:
nouns.check_overlap()

['Monday',
 'West Indies',
 'Ireland',
 'Asia Minor',
 'Western Islands',
 'Bahama',
 'Great Britain',
 'Europe',
 'China',
 'Sicily',
 'Isle of Wight',
 'Switzerland',
 'South Wales',
 'Russia',
 'America',
 'East Indies',
 'Italy',
 'United Kingdoms',
 'France',
 'Antigua',
 'Cape',
 'Atlantic',
 'Streights',
 'Channel',
 'North Seas',
 'Longtown',
 'Milton',
 'Priory',
 'Blair',
 'Butler',
 'America',
 'Bahama',
 'Antigua',
 'Europe',
 'Switzerland',
 'Isle of Wight',
 'West Indies',
 'Sicily',
 'Great Britain',
 'United Kingdoms',
 'South Wales',
 'China',
 'Ireland',
 'East Indies',
 'Italy',
 'France',
 'Asia Minor',
 'Western Islands',
 'Russia',
 'Atlantic',
 'Channel',
 'Streights',
 'North Seas',
 'Cape',
 'Monday',
 'Hilll',
 'Stokes',
 'Longtown',
 'Blair',
 'Oliver',
 'Brandon',
 'Edward',
 'Fitzwilliam',
 'Philips',
 'Butler',
 'Edward',
 'Fitzwilliam',
 'Oliver',
 'Brandon',
 'Augusta',
 'Frances',
 'Frances',
 'Williams',
 'Milton']

When I initially generated these proper noun types, the operation above returned only 5 or 6 entries.  However, that is because I did a fair amount of item assignment and reassignment that was written over instead of kept, so the current run through is not 100% equivalent to the originally generated types. However, this notebook is a good portrayal of my process - I offloaded as much assignment as I could think of onto functions and loops and searches, assigned the rest of the nouns manually, and doublechecked all noun type lists for accuracy. Although this was lot of manual assignment, in the end it took me only one afternoon to find and categorize with near 100% accuracy, over 800 unique proper nouns, in a corpus with a vocabulary of over 14000 words and a total length of over 700000 words. 

In [75]:
over_count, under_count = 0, 0
for names in nouns.ppn_types.values():
    for word in names: 
        if len(re.findall(word, clean_text)) >= 5:
            over_count += 1
        else:
            under_count += 1
print(f'Names that appear 5 or more time: {over_count}\nNames that appear less than 5 times: {under_count}')

Names that appear 5 or more time: 381
Names that appear less than 5 times: 624


In [ ]:
# nouns.save('proper_nouns_obj')
# nouns.save_types('proper_noun_types')

And finally, replace all these laboriously identified proper nouns with a key of the form '-\[A-Z]*3-' (eg -PLC- for places, and -LNM- for last names)

In [ ]:
text_min_clean = pp.load_data('text_by_par_min_clean')
text_with_placeholders = nouns.input_placeholders(text_min_clean)

In [ ]:
text_with_placeholders[:3]

In [ ]:
# pp.save_data(text_with_placeholders, 'text_with_placeholders')